In [17]:
import os
from llama_index.core import Settings
from llama_index.llms.groq import Groq
import dotenv

dotenv.load_dotenv()

True

In [18]:
Settings.llm = Groq(model="llama3-70b-8192",
                    api_key=os.getenv("groqKey"))
import pandas as pd

df_vendas = pd.read_csv("vendas.csv")

In [19]:
from llama_index.core import PromptTemplate
from llama_index.experimental.query_engine.pandas import PandasInstructionParser

In [20]:
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO.\n"
    "5. Não coloque a expressão entre aspas.\n")

# Prompt que será enviado ao modelo para que ela gere o código Pandas desejado
pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado `df_vendas`.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de `print(df_vendas.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)

# Prompt para guiar o modelo a sintetizar uma resposta com base nos resultados obtidos pela consulta Pandas
response_synthesis_prompt_str = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
   "Consulta: {query_str}\n\n"
   "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
   "Saída do Pandas: {pandas_output}\n\n"
   "Resposta:"
   "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}"
)

In [21]:
def descreve_colunas(df):
    descricao = "\n".join([f"{col}: {str(df[col].dtype)}"for col in df.columns])
    return "detalhes da colunas do dataframe:\n" , descricao

In [22]:
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str,colunas_detalhes=descreve_colunas(df_vendas),df_str=df_vendas.head(5)
)

pandas_output_parser = PandasInstructionParser(df_vendas)

response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)


llm = Groq(model="llama3-70b-8192",api_key=os.getenv("groqKey"))

In [23]:
from llama_index.core.query_pipeline import (QueryPipeline as QP,Link,InputComponent)

In [24]:
qp = QP(
    modules={
        "input":InputComponent(),
        "pandas_prompt":pandas_prompt,
        "llm1":llm,
        "pandas_output_parser":pandas_output_parser,
        "response_synthesis_prompt":response_synthesis_prompt,
        "llm2":llm
    },
    verbose=True
)

In [25]:
qp.add_chain(["input","pandas_prompt","llm1","pandas_output_parser"])
qp.add_links(
    [
        Link("input","response_synthesis_prompt",dest_key="query_str"),
        Link("llm1","response_synthesis_prompt",dest_key="pandas_instructions"),
        Link("pandas_output_parser","response_synthesis_prompt",dest_key="pandas_ouput")
    ]
)

qp.add_link("response_synthesis_prompt","llm2")

In [26]:
response = qp.run(query_str="qual e o vendedor com maior volume de vendas?")

> Running module input with input: 
query_str: qual e o vendedor com maior volume de vendas?

> Running module pandas_prompt with input: 
query_str: qual e o vendedor com maior volume de vendas?

> Running module llm1 with input: 
messages: Você está trabalhando com um dataframe do pandas em Python chamado `df_vendas`.
('detalhes da colunas do dataframe:\n', 'ID_compra: object\nfilial: object\ncidade: object\ntipo_cliente: object\ngenero...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Running module pandas_output_parser with input: 
input: assistant: df_vendas.groupby('filial')['total'].sum().idxmax()



Traceback (most recent call last):
  File "/home/lucas/Área de Trabalho/llm_curso/.venv/lib/python3.12/site-packages/llama_index/experimental/query_engine/pandas/output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lucas/Área de Trabalho/llm_curso/.venv/lib/python3.12/site-packages/llama_index/experimental/exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df_vendas' is not defined


In [27]:
import textwrap

texto = response

text_formatado = textwrap.fill(texto,width=100)
print(text_formatado)

There was an error running the output as Python code. Error message: name 'df_vendas' is not defined


In [28]:
# Pipeline de consulta
'''Função para obter uma descrição das colunas do DataFrame'''
def descricao_colunas(df):
    descricao = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Aqui estão os detalhes das colunas do dataframe:\n" + descricao

'''Definição de módulos da pipeline'''
def pipeline_consulta(df):
    instruction_str = (
        "1. Converta a consulta para código Python executável usando Pandas.\n"
        "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
        "3. O código deve representar uma solução para a consulta.\n"
        "4. IMPRIMA APENAS A EXPRESSÃO.\n"
        "5. Não coloque a expressão entre aspas.\n")

    pandas_prompt_str = (
        "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
        "{colunas_detalhes}\n\n"
        "Este é o resultado de `print(df.head())`:\n"
        "{df_str}\n\n"
        "Siga estas instruções:\n"
        "{instruction_str}\n"
        "Consulta: {query_str}\n\n"
        "Expressão:"
)

    response_synthesis_prompt_str = (
       "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
       "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
       "Consulta: {query_str}\n\n"
       "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
       "Saída do Pandas: {pandas_output}\n\n"
       "Resposta: \n\n"
       "Ao final, exibir o código usado em para gerar a resposta, no formato: O código utilizado foi `{pandas_instructions}`"
    )

    pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str,
    df_str=df.head(5),
    colunas_detalhes=descricao_colunas(df)
)

    pandas_output_parser = PandasInstructionParser(df)
    response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

    '''Criação do Query Pipeline'''
    qp = QP(
        modules={
            "input": InputComponent(),
            "pandas_prompt": pandas_prompt,
            "llm1": llm,
            "pandas_output_parser": pandas_output_parser,
            "response_synthesis_prompt": response_synthesis_prompt,
            "llm2": llm,
        },
        verbose=True,
    )
    qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
    qp.add_links(
        [
            Link("input", "response_synthesis_prompt", dest_key="query_str"),
            Link("llm1", "response_synthesis_prompt", dest_key="pandas_instructions"),
            Link("pandas_output_parser", "response_synthesis_prompt", dest_key="pandas_output"),
        ]
    )
    qp.add_link("response_synthesis_prompt", "llm2")
    return qp

### Interface com o Gradio

In [30]:
import gradio as gr

def load_data(file_path, df_state):
    if file_path is None or file_path == "":
        return "Faça o upload de um dataset para começar a análise", df_state
    try:
        df = pd.read_csv(file_path)
        return "Arquivo carregado com sucesso!", df
    except Exception as e:
        return f"Erro ao carregar o dataset: {str(e)}", df_state
    
def proccess_question(question, df_state):
    if df_state is not None and question:
        qp = pipeline_consulta(df_state)
        response = qp.run(query_str=question)
        return response.message.content
    return ""

with gr.Blocks() as app:
    in_file = gr.File(file_count="single",
                      type="filepath",
                      label="Upload CSV")
    status_upload = gr.Textbox(label="Status do Upload")
    in_question = gr.Textbox(label="Digite sua pergunta sobre os dados")
    bt_submit = gr.Button("Enviar")
    out_answer = gr.Textbox(label="Resposta")
    df_state = gr.State(value=None)
    
    in_file.change(fn=load_data,
                   inputs=[in_file, df_state],
                   outputs=[status_upload, df_state])
    
    bt_submit.click(fn=proccess_question,
                    inputs=[in_question, df_state],
                    outputs=out_answer)
app.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
INFO:httpx:HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
> Running module input with input: 
query_str: qual o produto que mais vendeu?

> Running module pandas_prompt with input: 
query_str: qual o produto que mais vendeu?

> Running module llm1 with input: 
messages: Você está trabalhando com um dataframe do pandas em Python chamado `df`.
Aqui estão os detalhes das colunas do dataframe:
`ID_compra`: object
`filial`: object
`cidade`: object
`tipo_cliente`: object
`...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Running module pandas_output_parser with input: 
input: assistant: df.groupby('tipo_produto')['quantidade'].sum().idxmax()

> Running module response_synthesis_prompt with input: 
query_str: qual o produto que mais vendeu?
pandas_instructions: as